# Phase 3: Optimal Temperature Tuning Model
## Personalized AI-Based Temperature Optimization for Therapeutic Hypothermia

**Objective:** Build a machine learning regression model that predicts the individually optimal neuroprotective temperature based on the infant's real-time physiological data.

**Key Components:**
- Temperature Gradient Prediction (next 1-hour temperature)
- Temperature Stability Assessment (minimize overshoot/undershoot)
- Individualized Temperature Recommendations
- Model Comparison (Random Forest, Gradient Boosting, Neural Network)

## Section 1: Import Libraries and Load Preprocessed Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load preprocessed data
print("Loading preprocessed dataset...")
df = pd.read_csv('../data/preprocessed_normalized_dataset.csv')

print(f"✓ Dataset loaded: {df.shape}")
print(f"Patients: {df['patient_id'].nunique()}")
print(f"Time range: {df['time_hours'].min():.1f} to {df['time_hours'].max():.1f} hours")

ModuleNotFoundError: No module named 'tensorflow'

## Section 2: Prepare Data for Temperature Prediction Model

In [ ]:
print("Preparing temperature prediction task...\n")

# Create target variable: temperature 1 hour in the future
df_model = df.copy().sort_values(['patient_id', 'time_hours']).reset_index(drop=True)

# Create future temperature (target for regression)
df_model['target_temp_next_1h'] = 0.0

for patient_id in df_model['patient_id'].unique():
    mask = df_model['patient_id'] == patient_id
    indices = df_model[mask].index
    
    # Shift temperature forward by 12 measurements (1 hour with 5-min intervals)
    future_temps = df_model.loc[indices, 'rectal_temperature_c'].shift(-12).values
    df_model.loc[indices, 'target_temp_next_1h'] = future_temps

# Remove last 12 rows of each patient (no future data)
df_model = df_model.dropna(subset=['target_temp_next_1h'])

# Select features for the model
feature_cols = [col for col in df_model.columns if col not in [
    'patient_id', 'time_minutes', 'time_hours', 'rectal_temperature_c', 
    'target_temp_c', 'hie_severity', 'target_temp_next_1h', 'baseline_patient_id',
    'baseline_birth_hour', 'baseline_core_temp'
] and not col.startswith('baseline_')]

# Add some baseline features for personalization
baseline_features = ['baseline_birth_weight_kg', 'baseline_gestational_age_weeks', 
                     'baseline_seizure_risk_factor']
feature_cols = feature_cols + baseline_features

print(f"Total features for temperature model: {len(feature_cols)}")
print(f"Training samples: {len(df_model):,}")

# Remove any remaining NaN values
df_model_clean = df_model[feature_cols + ['target_temp_next_1h', 'hie_severity']].dropna()

print(f"Clean samples after removing NaN: {len(df_model_clean):,}")

In [ ]:
# Prepare training and testing data
X = df_model_clean[feature_cols].values
y = df_model_clean['target_temp_next_1h'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData split:")
print(f"  Training set: {len(X_train):,} samples")
print(f"  Testing set: {len(X_test):,} samples")
print(f"  Feature dimensions: {X_train.shape[1]}")

print(f"\nTarget variable statistics:")
print(f"  Mean: {y.mean():.2f}°C")
print(f"  Std: {y.std():.2f}°C")
print(f"  Min: {y.min():.2f}°C")
print(f"  Max: {y.max():.2f}°C")

## Section 3: Train Multiple Regression Models

In [ ]:
print("Training temperature prediction models...\n")

# Dictionary to store models and results
models = {}
results = {}

# Model 1: Random Forest Regressor
print("1. Training Random Forest Regressor...")
rf_model = RandomForestRegressor(n_estimators=100, max_depth=15, 
                                 random_state=42, n_jobs=-1, verbose=0)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
rf_rmse = np.sqrt(rf_mse)
rf_r2 = r2_score(y_test, rf_pred)
rf_mae = mean_absolute_error(y_test, rf_pred)

models['Random Forest'] = rf_model
results['Random Forest'] = {
    'MSE': rf_mse, 'RMSE': rf_rmse, 'R2': rf_r2, 'MAE': rf_mae,
    'predictions': rf_pred
}
print(f"   ✓ RMSE: {rf_rmse:.4f}°C, R²: {rf_r2:.4f}, MAE: {rf_mae:.4f}°C")

# Model 2: Gradient Boosting Regressor
print("\n2. Training Gradient Boosting Regressor...")
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, 
                                     max_depth=5, random_state=42)
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)
gb_mse = mean_squared_error(y_test, gb_pred)
gb_rmse = np.sqrt(gb_mse)
gb_r2 = r2_score(y_test, gb_pred)
gb_mae = mean_absolute_error(y_test, gb_pred)

models['Gradient Boosting'] = gb_model
results['Gradient Boosting'] = {
    'MSE': gb_mse, 'RMSE': gb_rmse, 'R2': gb_r2, 'MAE': gb_mae,
    'predictions': gb_pred
}
print(f"   ✓ RMSE: {gb_rmse:.4f}°C, R²: {gb_r2:.4f}, MAE: {gb_mae:.4f}°C")

# Model 3: Neural Network (MLP)
print("\n3. Training Neural Network (MLP)...")
nn_model = MLPRegressor(hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001,
                       max_iter=500, random_state=42, early_stopping=True, 
                       validation_fraction=0.1, verbose=0)
nn_model.fit(X_train, y_train)
nn_pred = nn_model.predict(X_test)
nn_mse = mean_squared_error(y_test, nn_pred)
nn_rmse = np.sqrt(nn_mse)
nn_r2 = r2_score(y_test, nn_pred)
nn_mae = mean_absolute_error(y_test, nn_pred)

models['Neural Network'] = nn_model
results['Neural Network'] = {
    'MSE': nn_mse, 'RMSE': nn_rmse, 'R2': nn_r2, 'MAE': nn_mae,
    'predictions': nn_pred
}
print(f"   ✓ RMSE: {nn_rmse:.4f}°C, R²: {nn_r2:.4f}, MAE: {nn_mae:.4f}°C")

print("\n" + "=" * 80)
print("MODEL COMPARISON - TEMPERATURE PREDICTION (1-Hour Ahead)")
print("=" * 80)

comparison_df = pd.DataFrame({
    'Model': results.keys(),
    'RMSE (°C)': [results[m]['RMSE'] for m in results.keys()],
    'MAE (°C)': [results[m]['MAE'] for m in results.keys()],
    'R² Score': [results[m]['R2'] for m in results.keys()]
})
comparison_df = comparison_df.sort_values('RMSE (°C)')
print(comparison_df.to_string(index=False))

# Best model
best_model_name = comparison_df.iloc[0]['Model']
print(f"\n✓ Best performing model: {best_model_name}")

## Section 4: Model Performance Visualization

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Temperature Prediction Models - Performance Analysis', fontsize=16, fontweight='bold')

colors = ['#2ecc71', '#e74c3c', '#3498db']

# Plot 1: Actual vs Predicted comparison
ax = axes[0, 0]
for idx, (model_name, pred) in enumerate([(m, results[m]['predictions']) for m in results.keys()]):
    ax.scatter(y_test, pred, alpha=0.5, label=model_name, s=20, color=colors[idx])
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, label='Perfect Prediction')
ax.set_xlabel('Actual Temperature (°C)', fontsize=11, fontweight='bold')
ax.set_ylabel('Predicted Temperature (°C)', fontsize=11, fontweight='bold')
ax.set_title('Actual vs Predicted Temperatures', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: RMSE comparison
ax = axes[0, 1]
rmse_values = [results[m]['RMSE'] for m in results.keys()]
bars = ax.bar(results.keys(), rmse_values, color=colors, edgecolor='black', linewidth=1.5)
ax.set_ylabel('RMSE (°C)', fontsize=11, fontweight='bold')
ax.set_title('Root Mean Squared Error Comparison', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
for i, (bar, val) in enumerate(zip(bars, rmse_values)):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{val:.4f}', ha='center', va='bottom', fontweight='bold')

# Plot 3: R² Score comparison
ax = axes[1, 0]
r2_values = [results[m]['R2'] for m in results.keys()]
bars = ax.bar(results.keys(), r2_values, color=colors, edgecolor='black', linewidth=1.5)
ax.set_ylabel('R² Score', fontsize=11, fontweight='bold')
ax.set_title('R² Score Comparison', fontsize=12, fontweight='bold')
ax.set_ylim([0, 1.1])
ax.grid(True, alpha=0.3, axis='y')
for i, (bar, val) in enumerate(zip(bars, r2_values)):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{val:.4f}', ha='center', va='bottom', fontweight='bold')

# Plot 4: Prediction error distribution
ax = axes[1, 1]
for idx, (model_name, pred) in enumerate([(m, results[m]['predictions']) for m in results.keys()]):
    errors = y_test - pred
    ax.hist(errors, bins=30, alpha=0.5, label=model_name, color=colors[idx], edgecolor='black')
ax.set_xlabel('Prediction Error (°C)', fontsize=11, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax.set_title('Distribution of Prediction Errors', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
ax.axvline(x=0, color='k', linestyle='--', linewidth=1, alpha=0.7)

plt.tight_layout()
plt.savefig('../data/temperature_model_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Performance visualization saved")

## Section 5: Feature Importance Analysis

In [ ]:
# Extract feature importance from tree-based models
print("Analyzing feature importance...\n")

# Random Forest feature importance
rf_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Gradient Boosting feature importance
gb_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': gb_model.feature_importances_
}).sort_values('importance', ascending=False)

# Visualize top features
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('Feature Importance - Temperature Prediction Models', fontsize=14, fontweight='bold')

# Random Forest
ax = axes[0]
top_rf = rf_importance.head(12)
bars = ax.barh(range(len(top_rf)), top_rf['importance'].values, color='#2ecc71', edgecolor='black')
ax.set_yticks(range(len(top_rf)))
ax.set_yticklabels(top_rf['feature'].values)
ax.set_xlabel('Importance Score', fontsize=11, fontweight='bold')
ax.set_title('Random Forest - Top 12 Features', fontsize=12, fontweight='bold')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

# Gradient Boosting
ax = axes[1]
top_gb = gb_importance.head(12)
bars = ax.barh(range(len(top_gb)), top_gb['importance'].values, color='#e74c3c', edgecolor='black')
ax.set_yticks(range(len(top_gb)))
ax.set_yticklabels(top_gb['feature'].values)
ax.set_xlabel('Importance Score', fontsize=11, fontweight='bold')
ax.set_title('Gradient Boosting - Top 12 Features', fontsize=12, fontweight='bold')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../data/temperature_model_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("Top 10 Most Important Features (Random Forest):")
print(rf_importance.head(10).to_string(index=False))

print("\n✓ Feature importance analysis completed")

## Section 6: Save Models and Results

In [ ]:
import pickle
import json

# Save models
print("Saving models and results...\n")

# Save best performing model
best_model = models[best_model_name]
with open('../models/temperature_optimization_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
print(f"✓ Best model ({best_model_name}) saved")

# Save all models
for model_name, model in models.items():
    safe_name = model_name.lower().replace(' ', '_')
    with open(f'../models/temperature_model_{safe_name}.pkl', 'wb') as f:
        pickle.dump(model, f)

# Save results summary
results_summary = {
    'best_model': best_model_name,
    'comparison': {
        model: {
            'RMSE': float(results[model]['RMSE']),
            'MAE': float(results[model]['MAE']),
            'R2': float(results[model]['R2'])
        } for model in results.keys()
    },
    'test_set_size': len(y_test),
    'train_set_size': len(y_train),
    'feature_count': len(feature_cols)
}

with open('../models/temperature_model_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

# Save feature names for inference
with open('../models/temperature_model_features.json', 'w') as f:
    json.dump(feature_cols, f, indent=2)

print("✓ All models saved to ../models/")
print(f"✓ Results summary saved")
print(f"✓ Feature list saved")

print("\n" + "=" * 80)
print("PHASE 3 SUMMARY - TEMPERATURE OPTIMIZATION MODEL")
print("=" * 80)
print(f"\nBest Model: {best_model_name}")
print(f"  RMSE: {results[best_model_name]['RMSE']:.4f}°C")
print(f"  MAE: {results[best_model_name]['MAE']:.4f}°C")
print(f"  R² Score: {results[best_model_name]['R2']:.4f}")
print(f"\nModel predicts optimal temperature 1 hour ahead")
print(f"Training samples: {len(X_train):,}")
print(f"Features used: {len(feature_cols)}")
print(f"\nReady for Phase 4: Seizure and Complication Prediction")